In [79]:
using GLMakie, LinearAlgebra

include("boids.jl")

# TODO: Make boids not collided
# TODO: Make border limits for boids
# TODO: Try to make inifite run/stop animation
# TODO: Maybeee sliders for parameters?
# TODO: Maybeeeeee try make .exe 

# Make boids
width = height = 1000;
numBoids = 500
flock = [Boid(i, rand(-width:width, 3).*2, randn(3).*3, randn(3)) for i=1:numBoids]
points = Observable([Point3f(boid.position) for boid in flock])

# Parameters
alignValue = 0.5# ls1[1].value
separationValue = 1#ls2[1].value
cohesionValue = 0.6 #ls3[1].value

# Create figure
fig = Figure()

# create 3d axis
ax = Axis3(fig[1, 1], aspect = (1, 1, 1),
        title = "Flocking simulation")
        ax.perspectiveness = 0

# create button grid
fig[2, 1] = buttongrid = GridLayout(tellwidth = false)

sl1 = Slider(fig[1, 2], range = 0.01:0.01:5, horizontal = false, startvalue = 1)
sl2 = Slider(fig[1, 3], range = 0.01:0.01:5, horizontal = false, startvalue = 1)
sl3 = Slider(fig[1, 4], range = 0.01:0.01:5, horizontal = false, startvalue = 0.5)

# # # get slider values
alignValue = sl1.value
separationValue = sl2.value
cohesionValue = sl3.value

sliders = lift(sl1.value, sl2.value, sl3.value) do a, s, c
    alignValue = a
    separationValue = s
    cohesionValue = c
end

# alignValue = 0.7# ls1[1].value
# separationValue = 1#ls2[1].value
# cohesionValue = 1.5 #ls3[1].value

# Set limits
limits!(ax, -2width, 2*width, -2*width, 2*width, -2*width, 2*width)

labels = ["Run", "Step", "Restart"]
button_count = 3

# create buttons
buttons = buttongrid[1, 1:button_count] = [
    Button(fig,
        label = l, height = 30, width = 120, textsize = 30
    )
    for l in labels
]

# run button
on(buttons[1].clicks) do click
    frames = 1:100
    @async begin
    for i = frames
        for boid in flock
            fly!(boid, flock, alignValue[], separationValue[], cohesionValue[])
        end
        
        points[] = [Point3f(boid.position) for boid in flock]
        sleep(0.00001)
    end
    end
    # record(fig, "append_animation.mp4", frames;
    #         framerate = 30) do frame
    #     points[] += Point3f[(0., 0.01, 0.)]
    #     sleep(0.05)
    # end
end

# step button - the same as run
on(buttons[2].clicks) do click
    for boid in flock
        fly!(boid, flock, alignValue[], separationValue[], cohesionValue[])
    end
    
    points[] = [Point3f(boid.position) for boid in flock]
end

# reset button
on(buttons[3].clicks) do click
    flock = [Boid(i, rand(-width:width, 3).*2, randn(3).*3, randn(3)) for i=1:numBoids]
    points[] = [Point3f(boid.position) for boid in flock]
end

scatter!(ax, points, markersize=1000)

display(fig)

StackOverflowError: StackOverflowError: